# Dividends vs Growth

Comparing the approaches of investing for dividends or growth over time

In [4]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from tax_calculator import tax

# Allow plotly to operate within the notebook
init_notebook_mode()

%load_ext autoreload
%autoreload 2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
starting_capital = 1
capital_growth = 0.07 # 7%
dividend_payout = 0.04 # 4%


salary = 100000
num_years = 10

In [28]:
# Year 2
new_capital = (starting_capital*(1+dividend_payout))*(1+capital_growth)
new_capital

# tax(new_capital)

1.1128

In [51]:
def eoy_stats(year, salary, dividend_income, tax_paid, capital):

    return {
        'year': year,
        'salary': salary,
        'dividend_income': dividend_income,
        'tax_paid': tax_paid,
        'capital': capital
    }

In [52]:
# overall_capital = starting_capital
overall_capital = 10000
stat_history = []


for i in range(1,num_years):
    
    dividend_income = overall_capital*dividend_payout
    total_earnings = salary + dividend_income
    
    
    overall_capital = (overall_capital*(1+dividend_payout))*(1+capital_growth)
    capital_history.append(overall_capital)
    
    stat_history.append(eoy_stats(
        year = i,
        salary = salary,
        dividend_income = dividend_income,
        tax_paid = tax(total_earnings),
        capital = overall_capital
        ))

In [54]:
df = pd.DataFrame(stat_history)
df

,capital,dividend_income,salary,tax_paid,year
0,11128.000000,400.000000,100000,24645.000000,1
1,12383.238400,445.120000,100000,24661.694400,2
2,13780.067692,495.329536,100000,24680.271928,3
3,15334.459327,551.202708,100000,24700.945002,4
4,17064.186339,613.378373,100000,24723.949998,5
5,18989.026558,682.567454,100000,24749.549958,6
6,21130.988754,759.561062,100000,24778.037593,7
7,23514.564286,845.239550,100000,24809.738634,8
8,26167.007137,940.582571,100000,24845.015551,9


In [47]:
df.apply(lambda x: x['dividend_income'], axis = 1)

0     400.000000
1     445.120000
2     495.329536
3     551.202708
4     613.378373
5     682.567454
6     759.561062
7     845.239550
8     940.582571
9    1046.680285
dtype: float64

In [66]:
stat_history[-1]['capital']

26167.007136927077

In [64]:
total_capital_gains = sum([taxable_capital_gains(x['dividend_income'], (num_years - x['year'])) for x in stat_history])
print(total_capital_gains)

1057.6372692477576


In [61]:
def taxable_capital_gains(initial_investment, years_held):
    
    current_value = compound_interest(initial_investment, capital_growth, 1, years_held)
    capital_gain = current_value - initial_investment
    
    if(years_held > 1):
        return capital_gain/2
    else:
        return capital_gain

In [46]:
def compound_interest(principal, interest_rate, compounds_per_year, years):
        """Compound Interest Calculator
        
        Args:
            principal (float): Initial principal 
            interest_rate (float): Interest Rate % as a decimal
            compounds_per_year (int): Number of compound events per year
            years (float): Number of years to compount over
        
        Returns:
            float: Final return
        """

        return principal * (1 + interest_rate/compounds_per_year) ** (compounds_per_year * years)

In [152]:
from GrowthCalculator import GrowthCalculator

In [173]:
gc_liquidation = GrowthCalculator(
        salary = 100000,
        starting_investment = 10000,
        capital_growth = 0.07,
        dividend_payout = 0.02,
        years = 10
)

gc_liquidation.calculate_growth(final_year_liquidation=False)

df_with_liquidation = pd.DataFrame(gc_liquidation.investment_history)
df_with_liquidation

,capital,dividend_income,excess_tax,salary,year
0,10000.000000,0.000000,0.000000,100000,0
1,10914.000000,200.000000,74.000000,100000,1
2,11911.539600,218.280000,80.763600,100000,2
3,13000.254319,238.230792,88.145393,100000,3
4,14188.477564,260.005086,96.201882,100000,4
5,15485.304414,283.769551,104.994734,100000,5
6,16900.661237,309.706088,114.591253,100000,6
7,18445.381674,338.013225,125.064893,100000,7
8,20131.289559,368.907633,136.495824,100000,8
9,21971.289425,402.625791,148.971543,100000,9


In [174]:
gc_wo_liquidation = GrowthCalculator(
        salary = 100000,
        starting_investment = 10000,
        capital_growth = 0.02,
        dividend_payout = 0.07,
        years = 10
)

gc_wo_liquidation.calculate_growth(final_year_liquidation=False)

df_without_liquidation = pd.DataFrame(gc_wo_liquidation.investment_history)
df_without_liquidation

,capital,dividend_income,excess_tax,salary,year
0,10000.000000,0.000000,0.000000,100000,0
1,10914.000000,700.000000,259.000000,100000,1
2,11911.539600,763.980000,282.672600,100000,2
3,13000.254319,833.807772,308.508876,100000,3
4,14188.477564,910.017802,336.706587,100000,4
5,15485.304414,993.193429,367.481569,100000,5
6,16900.661237,1083.971309,401.069384,100000,6
7,18445.381674,1183.046287,437.727126,100000,7
8,20131.289559,1291.176717,477.735385,100000,8
9,21971.289425,1409.190269,521.400400,100000,9


In [175]:
print("Dividend Plan")
print(gc_wo_liquidation.summarise())

print('')
print('')

print("Growth Plan")
print(gc_liquidation.summarise())

Dividend Plan
{'Total Tax Paid': 3961.36, 'Total Earnings': 10706.37, 'Total Capital': 23979.47, 'Final Position': 20018.11}


Growth Plan
{'Total Tax Paid': 1131.82, 'Total Earnings': 3058.96, 'Total Capital': 23979.47, 'Final Position': 22847.65}
